In [1]:
import mlflow
from azureml.core import Workspace

ws = Workspace.from_config()

# mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())


In [2]:
experiment_name = 'mlflow-aml-deployment'
mlflow.set_experiment(experiment_name)

In [3]:
import mlflow.pyfunc
from sys import version_info

from pyimagesearch import config
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
import pip
import pickle
import cv2
import os
import tensorflow as tf
import sklearn

# Construct and save the model
PYTHON_VERSION = "{major}.{minor}.{micro}".format(major=version_info.major,
                                                  minor=version_info.minor,
                                                  micro=version_info.micro)
# artifacts = {
#     "model_path": "output"
# }

conda_env = {
    'channels': ['defaults'],
    'dependencies': [
      'python={}'.format(PYTHON_VERSION),
      'pip',
      {
        'pip': [
          'mlflow',
            'imutils',
            'opencv-python-headless',
            'pillow',
            'azureml-mlflow',
          'tensorflow=={}'.format(tf.__version__),
          'scikit-learn=={}'.format(sklearn.__version__),
        ],
      },
    ],
    'name': 'tf_env'
}


model_path = "model"
# object_detection = Object_Detection()
mlflow.pyfunc.save_model(path=model_path, data_path ='output',loader_module ='pyimagesearch.model_loader', code_path=['pyimagesearch'], conda_env=conda_env)

# Load the model in `python_function` format
# loaded_model = mlflow.pyfunc.load_model(model_path)


In [ ]:
#Testing the deployed service
#preparing data

from io import BytesIO
import json
import base64
ENCODING = 'utf-8'
test_image_paths =['dataset/images/airplane/image_0001.jpg','dataset/images/airplane/image_0002.jpg']
base64_string_list=[]
for test_image_path in test_image_paths:
    with open(test_image_path,"rb") as img:
        image_bytes = BytesIO(img.read())

    encoded_image =base64.b64encode(image_bytes.getvalue())
    base64_string = encoded_image.decode(ENCODING)
    base64_string= "b'{0}'".format(base64_string)
    base64_string_list.append(base64_string)

In [6]:
from mlflow.pyfunc.scoring_server import parse_json_input, _get_jsonable_obj

import json
# data = json.loads(body)['data']
out = parse_json_input(json_input=body, orient="split")


In [7]:


import mlflow
model_path = "model"
loaded_model = mlflow.pyfunc.load_model(model_path)

pred_output = loaded_model.predict(out)
print(pred_output)

In [9]:
from azureml.core.webservice import AciWebservice, Webservice

# Set the model path to the model folder created by your run

# Configure 
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                memory_gb=4, 
                                                tags={'method' : 'keras/tf'}, 
                                                description='aml-mlflow model',
                                                location='westus2')


In [10]:
import mlflow.azureml
(webservice,model) = mlflow.azureml.deploy( model_uri=model_path,
                      workspace=ws,
                      model_name='object_detection_model', 
                      service_name='aml-mlflow-test-v2', 
                      deployment_config=aci_config, 
                      tags=None, mlflow_home=None, synchronous=True)

webservice.wait_for_deployment(show_output=True)


Registering model object_detection_model


2021/01/19 10:49:34 INFO mlflow.azureml: Registered an Azure Model with name: `object_detection_model` and version: `19`
2021/01/19 10:49:37 INFO mlflow.azureml: Deploying an Azure Webservice with name: `aml-mlflow-test-v2`


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
SucceededACI service creation operation finished, operation "Succeeded"


In [11]:

# Calling webservice object via AML API
image_request = {"data": base64_string_list}
# payload.append(image_request)

body = json.dumps(image_request)
response = webservice.run(body)
print(response)

[[0.1248638927936554, 0.1821192502975464, 0.8843519687652588, 0.834100604057312, 'airplane'], [0.14209172129631042, 0.1815202832221985, 0.8583660125732422, 0.8296916484832764, 'airplane']]


In [12]:
#another way to call http request directly 
import requests

response = requests.post(url=webservice.scoring_uri, data=body,headers={"Content-type": "application/json"})
print(response.text)


[[0.1248638927936554, 0.1821192502975464, 0.8843519687652588, 0.834100604057312, "airplane"], [0.14209172129631042, 0.1815202832221985, 0.8583660125732422, 0.8296916484832764, "airplane"]]


In [25]:
#Visualize result
import imutils
import pickle
import cv2
import os
import ast
pred_results = ast.literal_eval(response.text)
for pred_result, imagePath in zip( pred_results,test_image_paths):
	# load the input image (in Keras format) from disk and preprocess
	# it, scaling the pixel intensities to the range [0, 1]

	startX, startY, endX, endY,label = pred_result[0],pred_result[1],pred_result[2],pred_result[3],pred_result[4]

	# determine the class label with the largest predicted
	# probability

	# load the input image (in OpenCV format), resize it such that it
	# fits on our screen, and grab its dimensions
	image = cv2.imread(imagePath)
	image = imutils.resize(image, width=600)
	(h, w) = image.shape[:2]

	# scale the predicted bounding box coordinates based on the image
	# dimensions
	startX = int(startX * w)
	startY = int(startY * h)
	endX = int(endX * w)
	endY = int(endY * h)

	# draw the predicted bounding box and class label on the image
	y = startY - 10 if startY - 10 > 10 else startY + 10
	cv2.putText(image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX,
		0.65, (0, 255, 0), 2)
	cv2.rectangle(image, (startX, startY), (endX, endY),
		(0, 255, 0), 2)

	# show the output image
	cv2.imshow("Output", image)
	cv2.waitKey(0)